In [1]:
pip install --upgrade openai python-dotenv pandas tqdm tenacity

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 98.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.6/364.6 kB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 55.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 19.9 MB/s eta 0:00:00
     ━━━

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
import json
import pandas as pd
from pathlib import Path
from datetime import datetime

from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from openai import OpenAI


In [5]:
# Load the .env file
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    print("python-dotenv not installed or .env missing")

# Fetch the key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("OPENAI_API_KEY loaded:", "YES" if OPENAI_API_KEY else "NO")


OPENAI_API_KEY loaded: YES


In [6]:
if not OPENAI_API_KEY:
    raise ValueError("❌ API Key not found. Make sure it's in your .env file as OPENAI_API_KEY.")

client = OpenAI(api_key=OPENAI_API_KEY)

# Tiny check call
try:
    resp = client.responses.create(
        model="gpt-4.1-mini",
        input="Hello! Just checking if the API key works."
    )
    print("API Key check successful ✔️")
except Exception as e:
    print("❌ Key failed:", e)


API Key check successful ✔️


In [7]:
# List all available OpenAI models with categories

from openai import OpenAI
import pandas as pd

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

models = client.models.list()

model_data = []

for m in models.data:
    model_id = m.id

    # Categorize based on name patterns
    if "gpt" in model_id.lower() or "o1" in model_id.lower():
        category = "Chat / Reasoning Models"
    elif "embed" in model_id.lower():
        category = "Embedding Models"
    elif "tts" in model_id.lower() or "audio" in model_id.lower():
        category = "Audio / Speech Models"
    elif "vision" in model_id.lower() or "clip" in model_id.lower():
        category = "Vision Models"
    else:
        category = "Other"

    model_data.append({
        "model_id": model_id,
        "category": category
    })

df_models = pd.DataFrame(model_data).sort_values("model_id")
df_models.reset_index(drop=True, inplace=True)

df_models


,model_id,category
0,babbage-002,Other
1,chatgpt-4o-latest,Chat / Reasoning Models
2,codex-mini-latest,Other
3,dall-e-2,Other
4,dall-e-3,Other
...,...,...
97,tts-1,Audio / Speech Models
98,tts-1-1106,Audio / Speech Models
99,tts-1-hd,Audio / Speech Models
100,tts-1-hd-1106,Audio / Speech Models


In [8]:
# Check if o3-mini is available in YOUR account and set MODEL_ID accordingly

want = "o3-mini"
has_o3_mini = (df_models["model_id"].str.contains(r"\bo3[-_]mini\b", case=False, na=False)).any()

if has_o3_mini:
    o3_row = df_models[df_models["model_id"].str.contains(r"\bo3[-_]mini\b", case=False, na=False)].sort_values("model_id").head(1)
    MODEL_ID = o3_row.iloc[0]["model_id"]
    print(f"✅ Found {MODEL_ID} in your account. Using this model.")
else:
    # sensible fallback if you don't have access
    MODEL_ID = "gpt-4.1-mini"
    print("⚠️ o3-mini not found on this account. Falling back to:", MODEL_ID)

MODEL_ID


✅ Found o3-mini in your account. Using this model.


'o3-mini'

In [9]:
MODEL_ID = "o3-mini"
print("Model selected:", MODEL_ID)


Model selected: o3-mini


In [10]:
from openai import OpenAI
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "❌ Missing OPENAI_API_KEY"

client = OpenAI(api_key=OPENAI_API_KEY)

try:
    resp = client.responses.create(
        model=MODEL_ID,
        input="Quick hello — please reply with 'OK' if you can hear me."
    )
    print("o3-mini check OK ✅")
    print("Sample output:", resp.output_text[:120].replace("\n"," "))
except Exception as e:
    print("❌ Model check failed:", e)


o3-mini check OK ✅
Sample output: OK


In [24]:
def build_user_prompt(title: str, body: str) -> str:
    return f"""
You are a trauma-informed, non-judgmental peer-support assistant who responds to posts where the author is unsure if an experience counts as sexual abuse. 
Your tone must always be: gentle, validating, warm, and never blaming.

You must avoid:
- legal advice
- medical advice

Your response MUST:
1. If the person got assaulted and they were confused about it, Explicitly mention that they got assaulted
2. Validate the person's emotions and confusion
3. Normalize uncertainty and mixed feelings
4. Offer gentle perspective about consent and boundaries
5. Gently suggest support options (trusted friend, partner, counselor, hotline), WITHOUT pressure
6. Be 50-120 words long
7. Use a warm, gentle, shame-free tone. No judgment, no criticism.
8. Encourage self-compassion (one sentence reminding them they deserve care and clarity).
9. Do not imply urgency unless the post states immediate danger.
10. Use clear, simple, human language. No clinical jargon.

Now respond to the following post:

TITLE: {title}

BODY:
{body}
"""


In [25]:
def generate_reply(title: str, body: str) -> str:
    # Build the full prompt (since Responses API does not accept system=)
    full_prompt = build_user_prompt(title, body)

    # Make the model call
    resp = client.responses.create(
        model=MODEL_ID,
        input=full_prompt,
    )

    # Extract model output text
    return resp.output_text.strip()


In [26]:
TEST_TITLE = "I don’t know if what happened was assault"
TEST_BODY = (
    "I was with someone I trust and things escalated. I didn’t say no, but I also didn’t really want it. "
    "Now I feel confused and guilty calling it assault. Part of me thinks I’m overreacting, but I can’t stop thinking about it."
)

reply = generate_reply(TEST_TITLE, TEST_BODY)
print(reply)


It sounds like what you experienced does fit the description of assault, especially since you didn’t feel comfortable even if you didn’t verbally say no. Your confusion and guilt are totally understandable—these mixed feelings often happen when boundaries aren’t clearly honored. Consent should always feel safe and free, and your feelings matter. Please know that you deserve care and clarity. It might help to talk with someone you trust, a supportive friend, or a counselor who understands. Remember, you deserve compassion and care in processing these feelings. You are worthy of feeling safe and heard.


In [27]:
TEST_TITLE = "Can I consider myself a survivor?"
TEST_BODY = (
    "I was wondering if I can consider myself part of the survivor community even if my experience wasn’t necessarily forced/violent? My traumatic experience was medical related and I gave consent but I was in a very distressed state when I said yes and they did not stop immediately when I told them to for medical reasons, as a result I do have sexual related trauma from this experience. I would like to call myself a survivor but I’m worried that using the label even to myself would be insensitive or offensive to other people and women who have experienced more violent/nonconsensual situations. "
)

reply = generate_reply(TEST_TITLE, TEST_BODY)
print(reply)


It sounds like what you experienced was a form of sexual assault—even though you initially said yes, being in a distressed state means your consent wasn’t fully informed or freely given, and that boundary wasn’t respected. It’s completely normal to feel confused and to question how you label your experience. Your feelings are valid, and it's okay to honor your journey by calling yourself a survivor if that resonates with you. Please consider talking with someone you trust or a supportive counselor whenever you feel ready. Remember, you deserve care, clarity, and compassion during this time.
